# Machine Learning Project

- Competition: [Kaggle](https://www.kaggle.com/c/to-loan-or-not-to-loan-that-is-the-question-ac2122/leaderboard)

In [1]:
# configuring pandas
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Utilitary functions
from datetime import date

# Format: YYMMDD -> Year is before 2000
def parse_date(d):
    year = int(str(d)[0:2])
    month = int(str(d)[2:4])
    day = int(str(d)[4:6])
    return year, month, day

def calculate_age(year, month, day):
    today = date.today()
    return today.year - year - ((today.month, today.day) < (month, day))

def parse_client_dates(df):
    ages = []
    genders = []
    birthdates = []

    for birth_number in df["birth_number"]:
        year = int(str(birth_number)[0:2])
        month = int(str(birth_number)[2:4])
        day = int(str(birth_number)[4:6])
        # check if the client is female or male
        if month > 12:
            genders.append(-1)  # female
            month = month - 50
        else:
            genders.append(1)  # male
        
        year = year + 1900

        ages.append(calculate_age(year, month, day))
        birthdates.append("%s-%s-%s" % (year, month, day))

    # Add the new columns to the dataframe
    df["client_age"] = ages
    df["client_gender"] = genders
    df["birthdate"] = birthdates
    df["birthdate"] = pd.to_datetime(df['birthdate'], format='%Y-%m-%d')

    # Drop the unnecessary column
    df = df.drop(columns="birth_number", axis=1)
    return df

In [3]:
def prep_loans(data_type):
    # process loans data
    df = pd.read_csv("data/loan_" + data_type + ".csv", sep=";", parse_dates=["date"])
    df.rename(columns={'date': 'loan_date','amount':'loan_amount'}, inplace=True)

    return df

In [4]:
from numpy import array
from sklearn.preprocessing import LabelEncoder

def prep_accounts():
    # Add the account data
    df = pd.read_csv("data/account.csv", sep=";", parse_dates=["date"])

    df.rename(columns={"date": "account_date", "district_id": "account_district_id"}, inplace=True)

    label_encoder = LabelEncoder()

    frequency = df["frequency"]
    values = array(frequency)
    integer_encoded = label_encoder.fit_transform(values)
    df["frequency"] = integer_encoded
    
    return df

In [5]:
def prep_dispositions():
     # to join the dispositions table we could count, for each account, the number of disponents
    dispositions_df = pd.read_csv("data/disp.csv", sep=";")
    dispositions_df.rename(columns={"client_id": "disp_client_id", "type": "disp_type"}, inplace=True)

    return dispositions_df

In [6]:
def prep_owners():
    # to join the dispositions table we could count, for each account, the number of disponents
    dispositions_df = pd.read_csv("data/disp.csv", sep=";")
    # disponents = dispositions_df.groupby('account_id')["type"].count().reset_index(name='account_clients')
    # add the owner id
    owners = dispositions_df[dispositions_df.type == "OWNER"][["account_id", "client_id"]].rename(columns={"client_id": "owner_id"})

    return owners

In [7]:
def prep_clients():
    # join clients data
    clients_df = pd.read_csv("data/client.csv", sep=";")
    clients_df = parse_client_dates(clients_df).rename(columns={"client_id": "owner_id", "district_id": "owner_district_id"})

    return clients_df

In [8]:
def prep_demographic(prefix):
    demographic_df = pd.read_csv("data/district.csv", sep=";")
    demographic_df = demographic_df.rename(columns=lambda x: x.strip())    

    label_encoder = LabelEncoder()

    demographic_df = demographic_df.drop(columns="name", axis=1)

    regions = demographic_df["region"]
    values = array(regions)
    integer_encoded = label_encoder.fit_transform(values)
    demographic_df["region"] = integer_encoded

    demographic_df["unemploymant rate '95"] = np.where(demographic_df["unemploymant rate '95"] == '?', demographic_df["unemploymant rate '96"], demographic_df["unemploymant rate '95"])
    demographic_df["no. of commited crimes '95"] = np.where(demographic_df["no. of commited crimes '95"] == '?', demographic_df["no. of commited crimes '96"], demographic_df["no. of commited crimes '95"])

    demographic_df = demographic_df.rename(columns={"code": "district_id"})

    return demographic_df.rename(columns=lambda x: prefix + x) 

In [9]:
def prep_transactions(data_type):
    df = pd.read_csv("data/trans_" + data_type + ".csv", sep=";", parse_dates=["date"], low_memory=False)
    
    df.loc[df["operation"].isna(),"operation"] = df.loc[df["operation"].isna(), "k_symbol"]
    
    label_encoder = LabelEncoder()

    operations = df["operation"]
    values = array(operations)
    integer_encoded = label_encoder.fit_transform(values)
    df["operation"] = integer_encoded

    df.loc[df["type"] == "withdrawal in cash", "type"] = "withdrawal"
    df.loc[df["type"] == "withdrawal", "amount"] *=-1 

    df = df.drop(['bank', 'account', 'k_symbol'], axis=1)
    
    df.rename(columns={"date": "transaction_date"}, inplace=True)

    return df

In [10]:
def prep_cards(data_type):
    df = pd.read_csv("data/card_" + data_type + ".csv", sep=";", parse_dates=["issued"])

    return df.rename(columns={"type": "card_type", "issued": "card_issued"})

In [11]:
def read_prep_data(data_type):
    # Read and process data
    loans_df = prep_loans(data_type)
    accounts_df = prep_accounts()
    owners_df = prep_owners()
    clients_df = prep_clients()
    owner_districts_df = prep_demographic("owner_")
    # account_districts_df = prep_demographic("account_")
    transactions_df = prep_transactions(data_type)

    df = pd.merge(loans_df, owners_df, on="account_id")
    df = pd.merge(df, clients_df, on="owner_id")
    df = pd.merge(df, accounts_df, on="account_id")
    df = pd.merge(df, transactions_df, on="account_id")
    df = pd.merge(df, owner_districts_df, on="owner_district_id")

    return df

In [12]:
# Methods for aggregation
def age_days(x):
    return (x.max()-x.min()).days

def abs_min(x):
    return x.abs().min()
def rangev(x):
    return x.max() - x.min()

def count_out(x):
    return sum(x=="withdrawal")
def count_in(x):
    return sum(x=="credit")

def mean_out(x):
    return np.mean(x=="withdrawal")
def mean_in(x):
    return np.mean(x=="credit")

def cov_out(x):
    return np.cov(x=="withdrawal")
def cov_in(x):
    return np.cov(x=="credit")

def std_in(x):
    return np.std(x=="credit")
def std_out(x):
    return np.std(x=="withdrawal")


def count_classic(x):
    return sum(x == "classic")
def count_junior(x):
    return sum(x == "junior")
def count_gold(x):
    return sum(x == "gold")

In [13]:
# Data Aggregation

def aggregate_transaction_data(df):
    new_columns = ["loan_id", "account_id", "status", "loan_date", 
                "loan_amount", "duration", "payments", 
                "owner_district_id", "client_age",
                "client_gender", "account_district_id", "frequency",
                "account_date", "birthdate"]

    # need to add covariances somehow
    df = df.groupby(new_columns, as_index=False, group_keys=False).agg({
        "transaction_date" : ["max", "min"],
        "operation": ["count"],
        "amount": ["mean","min","max","std","last", abs_min, rangev],
        "balance": ["mean","min","max","std","last", abs_min, rangev],
        "type": [count_in, count_out, mean_in, mean_out]
    })
    df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]
    
    # retreive some interesting data such as the client's age when the loan was issued

    return df

In [14]:
def aggregate_cards_data(data_type, df):
    df = pd.merge(df, prep_dispositions(), on="account_id", how="left")
    df = pd.merge(df, prep_cards(data_type), on="disp_id", how="left")

    new_columns = [x for x in list(df) if x not in ["card_id", "card_type", "disp_id", "disp_type", "disp_client_id", "card_issued"]]

    df = df.groupby(new_columns, as_index=False, group_keys=False).agg({
        "disp_id": ["count"],
        "card_type": [count_gold, count_classic, count_junior]
    })
    df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]

    return df

In [15]:
# Process Dates (dates need to be addressed later, they are lowering the prediction score)

def process_dates(df):
    df["loan_year"] = df["loan_date"].map(lambda x: x.year)
    df["account_year"] = df["account_date"].map(lambda x: x.year)
    df["transaction_year_max"] = df["transaction_date_max"].map(lambda x: x.year)
    df["transaction_year_min"] = df["transaction_date_min"].map(lambda x: x.year)
    
    return df.drop(['loan_date'], axis=1)

In [16]:
def aggregate_demographic_data(df):
    demographic_columns = ["district_id", "region", "no. of inhabitants",
                "no. of municipalities with inhabitants < 499",
                "no. of municipalities with inhabitants 500-1999",
                "no. of municipalities with inhabitants 2000-9999",
                "no. of municipalities with inhabitants >10000",
                "no. of cities", "ratio of urban inhabitants",
                "average salary", "unemploymant rate '95", 
                "unemploymant rate '96",
                "no. of enterpreneurs per 1000 inhabitants", 
                "no. of commited crimes '95", "no. of commited crimes '96"]
    
    # for now
    to_ignore = ["no. of municipalities with inhabitants < 499",
                "no. of municipalities with inhabitants 500-1999",
                "no. of municipalities with inhabitants 2000-9999",
                "no. of municipalities with inhabitants >10000",
                "no. of cities", "ratio of urban inhabitants",]

    demographic_df = prep_demographic("account_")
    

    return pd.merge(df, demographic_df[["account_{0}".format(x) for x in demographic_columns if x not in to_ignore]], on="account_district_id")
    

In [17]:
# Train pipeline
train_df = read_prep_data("train")
train_df = aggregate_transaction_data(train_df)
train_df = aggregate_cards_data("train", train_df)

train_df.loc[train_df["status"] == 1, "status"] = 0
train_df.loc[train_df["status"] == -1, "status"] = 1

# train_df = aggregate_demographic_data(train_df)
# train_df = process_dates(train_df)

# we should sort by loan data to have a realistic train/test split while training
train_df.sort_values(by=['loan_date'], inplace=True)

print("Train data")
train_df

Train data


,loan_id,account_id,status,loan_date,loan_amount,duration,payments,owner_district_id,client_age,client_gender,account_district_id,frequency,account_date,birthdate,transaction_date_max,transaction_date_min,operation_count,amount_mean,amount_min,amount_max,amount_std,amount_last,amount_abs_min,amount_rangev,balance_mean,balance_min,balance_max,balance_std,balance_last,balance_abs_min,balance_rangev,type_count_in,type_count_out,type_mean_in,type_mean_out,disp_id_count,card_type_count_gold,card_type_count_classic,card_type_count_junior
38,5314,1787,1,1993-07-05,96396,12,8033,30,74,-1,30,2,1993-03-22,1947-07-22,1993-06-20,1993-03-22,4,5025.000000,1100.0,9900.0,3774.806838,3300.0,1100.0,8800.0,12250.000000,1100.0,20100.0,8330.866301,20100.0,1100.0,19000.0,4,0,1.000000,0.000000,1,0,0,0
39,5316,1801,0,1993-07-11,165960,36,4610,46,53,1,46,1,1993-02-13,1968-07-22,1993-07-09,1993-02-13,37,1411.051351,-54300.0,36574.0,17882.999059,-3419.0,2.9,90874.0,52083.859459,700.0,120512.8,29122.059454,52208.9,700.0,119812.8,17,20,0.459459,0.540541,1,0,0,0
260,6863,9188,0,1993-07-28,127080,60,2118,45,85,1,45,1,1993-02-08,1936-06-02,1993-07-21,1993-02-08,24,844.708333,-14800.0,19065.0,8272.421583,-12000.0,48.6,33865.0,30060.954167,800.0,49590.4,11520.184451,20272.8,800.0,48790.4,15,9,0.625000,0.375000,1,0,0,0
41,5325,1843,0,1993-08-03,105804,36,2939,14,81,-1,12,1,1993-01-30,1940-04-20,1993-07-31,1993-01-30,25,1371.704000,-15600.0,26448.0,11376.124631,-14.6,14.6,42048.0,41297.480000,1000.0,65898.5,14151.260443,34292.7,1000.0,64898.5,13,12,0.520000,0.480000,1,0,0,0
315,7240,11013,0,1993-09-06,274740,60,4579,63,43,1,1,2,1993-02-14,1978-09-07,1993-08-31,1993-02-14,27,1522.707407,-36700.0,63366.0,27294.178010,182.8,30.0,100066.0,57188.211111,600.0,122893.1,25256.665817,41142.9,600.0,122293.1,13,14,0.481481,0.518519,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,6818,9030,0,1996-12-12,155616,48,3242,72,50,-1,72,1,1995-01-21,1971-04-10,1996-12-11,1995-01-21,172,352.872093,-36960.0,26388.0,8611.713428,-14600.0,14.6,63348.0,44197.509884,200.0,75294.1,11044.494392,60694.1,200.0,75094.1,70,102,0.406977,0.593023,2,0,0,0
86,5625,3189,1,1996-12-15,222180,60,3703,29,82,1,29,1,1995-11-29,1939-03-20,1996-12-10,1995-11-29,59,1009.813559,-52600.0,44352.0,18212.754147,-6900.0,14.6,96952.0,55230.444068,800.0,130659.1,26510.559286,59578.8,800.0,129859.1,28,31,0.474576,0.525424,1,0,0,0
251,6805,8972,0,1996-12-21,45024,48,938,70,64,-1,70,1,1996-05-21,1957-05-04,1996-12-17,1996-05-21,39,984.215385,-22100.0,31636.5,12758.414420,-17800.0,14.6,53736.5,41994.907692,800.0,63659.3,13151.510254,38384.3,800.0,62859.3,20,19,0.512821,0.487179,1,0,0,0
313,7233,10963,0,1996-12-25,115812,36,3217,16,68,1,16,1,1995-05-20,1953-06-01,1996-12-18,1995-05-20,124,337.726613,-50800.0,49887.0,19850.968193,-3100.0,14.6,100687.0,56646.516129,1100.0,119527.2,21971.162852,41878.1,1100.0,118427.2,51,73,0.411290,0.588710,1,0,0,0


In [18]:
# from lets_plot.bistro import corr
# # correlation matrix
# corr.corr_plot(train_df).points().build()

In [19]:
unwanted_features = ["status", "loan_id", "loan_date", "account_date", "transaction_date_max", "transaction_date_min", "birthdate"]
features = [x for x in list(train_df) if x not in unwanted_features]
target = "status"

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn import metrics
from sklearn import svm
import xgboost as xgb
from scipy.stats import uniform, randint

X = train_df[features]
y = train_df[target]

data_dmatrix = xgb.DMatrix(data=X,label=y)

# ter em conta o tempo neste split (os dados de teste são de datas posteriories aos dados de treino)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = xgb.XGBClassifier(objective="binary:logistic",use_label_encoder=False, eval_metric="auc")

params = {
          'colsample_bytree': uniform(0.7, 0.3),
          'learning_rate': uniform(0.03, 0.3),
          'max_depth': randint(2, 6),
          'alpha': uniform(0, 0.1),
          'n_estimators': randint(150, 350),
          'gamma': uniform(0, 0.1),
          'subsample': uniform(0.7, 0.3)
         }

clf = RandomizedSearchCV(model, 
                        param_distributions=params, 
                        random_state=123, 
                        n_iter=200, 
                        cv=4, 
                        scoring = 'roc_auc', 
                        error_score = 0, 
                        verbose=1, 
                        n_jobs=-1, 
                        return_train_score=True)

clf.fit(X, y)

print("Best AUC: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

# Predict the response for test dataset
y_pred = clf.predict_proba(X_test)[:, -1]

# Area Under the Curve, the higher the better
auc = metrics.roc_auc_score(y_test, y_pred)
print("AUC Score: ", auc)

Fitting 4 folds for each of 200 candidates, totalling 800 fits
Best AUC:  0.8423224955795379
Best Params:  {'alpha': 0.09865798891004533, 'colsample_bytree': 0.8134362793920771, 'gamma': 0.03821091929511316, 'learning_rate': 0.04533789016100603, 'max_depth': 5, 'n_estimators': 302, 'subsample': 0.7047236311939725}
AUC Score:  1.0


In [21]:
# Test Pipeline
test_df = read_prep_data("test")

test_df.status = test_df.status.fillna('')

test_df = aggregate_transaction_data(test_df)
test_df = aggregate_cards_data("test", test_df)
# test_df = aggregate_demographic_data(test_df)
# test_df = process_dates(test_df)

print("Test Data")
test_df

Test Data


,loan_id,account_id,status,loan_date,loan_amount,duration,payments,owner_district_id,client_age,client_gender,account_district_id,frequency,account_date,birthdate,transaction_date_max,transaction_date_min,operation_count,amount_mean,amount_min,amount_max,amount_std,amount_last,amount_abs_min,amount_rangev,balance_mean,balance_min,balance_max,balance_std,balance_last,balance_abs_min,balance_rangev,type_count_in,type_count_out,type_mean_in,type_mean_out,disp_id_count,card_type_count_gold,card_type_count_classic,card_type_count_junior
0,4962,25,,1997-12-08,30276,12,2523,68,59,1,68,1,1996-07-28,1962-02-09,1997-12-06,1996-07-28,164,192.816463,-62900.0,49734.0,19772.510052,-1110.0,14.6,112634.0,62991.264634,900.0,119652.7,22476.347980,31621.9,900.0,118752.7,51,113,0.310976,0.689024,1,0,0,0
1,4967,37,,1998-10-14,318480,60,5308,20,69,1,20,1,1997-08-18,1952-08-26,1998-10-12,1997-08-18,116,141.102586,-43200.0,47785.0,14052.393688,-11200.0,0.4,90985.0,39953.970690,-1011.2,104761.5,23451.647180,16367.8,900.0,105772.7,32,84,0.275862,0.724138,1,0,0,0
2,4968,38,,1998-04-19,110736,48,2307,19,81,-1,19,2,1997-08-08,1940-01-30,1998-04-10,1997-08-08,55,680.303636,-16800.0,20762.0,7425.399127,-231.0,30.0,37562.0,31383.449091,13841.0,55991.1,10950.811533,37416.7,13841.0,42150.1,23,32,0.418182,0.581818,1,0,0,0
3,4986,97,,1997-08-10,102876,12,8573,74,79,1,74,1,1996-05-05,1942-01-28,1997-08-07,1996-05-05,118,343.358475,-18600.0,26892.0,8464.912170,-15.0,3.0,45492.0,33057.374576,900.0,60334.7,12670.953546,40516.3,900.0,59434.7,33,85,0.279661,0.720339,2,0,0,0
4,4988,103,,1997-12-06,265320,36,7370,44,54,1,44,1,1996-03-10,1967-09-21,1997-11-30,1996-03-10,83,240.703614,-53500.0,48308.0,14986.841807,17.5,4.4,101808.0,46266.574699,379.0,116038.4,28827.091844,19992.8,379.0,115659.4,42,41,0.506024,0.493976,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,7279,11244,,1997-12-11,155760,24,6490,33,73,-1,33,1,1997-01-12,1948-07-31,1997-12-08,1997-01-12,83,668.038554,-47600.0,47085.0,18175.977109,-30300.0,14.6,94685.0,51964.019277,200.0,108538.5,24707.291006,55447.1,200.0,108338.5,37,46,0.445783,0.554217,1,0,0,0
350,7286,11271,,1997-01-31,67320,36,1870,5,49,1,5,1,1995-09-20,1972-01-26,1997-01-12,1995-09-20,111,177.998198,-11500.0,16833.0,5738.565617,1200.0,14.6,28333.0,26491.780180,200.0,43949.4,7217.696103,19757.5,200.0,43749.4,38,73,0.342342,0.657658,1,0,0,0
351,7292,11317,,1998-11-22,317460,60,5291,50,45,1,50,1,1997-07-11,1975-11-20,1998-11-21,1997-07-11,145,527.644828,-61700.0,49407.0,22093.100840,38736.0,14.6,111107.0,64978.072414,700.0,135890.4,26748.860366,76508.5,700.0,135190.4,50,95,0.344828,0.655172,1,0,0,0
352,7294,11327,,1998-09-27,39168,24,1632,7,42,-1,7,1,1997-10-15,1979-02-22,1998-09-13,1997-10-15,54,1150.357407,-23600.0,30462.0,12230.599576,20308.0,14.6,54062.0,55438.851852,300.0,81495.6,14622.737828,62119.2,300.0,81195.6,23,31,0.425926,0.574074,1,0,0,0


In [22]:
X_test = test_df[features]

probs = clf.predict_proba(X_test)

test_df[target] = probs
submission = test_df[["loan_id", "status"]]
submission = submission.rename(columns={"loan_id": "Id", "status": "Predicted"})

submission.to_csv("results.csv", index=False)

submission

,Id,Predicted
0,4962,0.995554
1,4967,0.089863
2,4968,0.990019
3,4986,0.987825
4,4988,0.314416
...,...,...
349,7279,0.987256
350,7286,0.972865
351,7292,0.995408
352,7294,0.997744
